# 1.实现sift（调接口）

In [1]:
import cv2
import numpy as np

img = cv2.imread("gray_test.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SIFT_create()
keypoints, descriptor = sift.detectAndCompute(gray, None)

# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS对图像的每个关键点都绘制了圆圈和方向。
img = cv2.drawKeypoints(image=img, outImage=img, keypoints=keypoints,
                        flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS,
                        color=(51, 163, 236))
                        
#img=cv2.drawKeypoints(gray,keypoints,img)

cv2.imshow('sift_keypoints', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2.实现ransac 

In [2]:
import numpy as np
import scipy as sp
import scipy.linalg as sl
 
def ransac(data, model, n, k, t, d, debug = False, return_all = False):
    """
    输入:
        data - 样本点
        model - 假设模型:事先自己确定
        n - 生成模型所需的最少样本点
        k - 最大迭代次数
        t - 阈值:作为判断点满足模型的条件
        d - 拟合较好时,需要的样本点最少的个数,当做阈值看待
    输出:
        bestfit - 最优拟合解（返回nil,如果未找到）
    
    iterations = 0
    bestfit = nil #后面更新
    besterr = something really large #后期更新besterr = thiserr
    while iterations < k 
    {
        maybeinliers = 从样本中随机选取n个,不一定全是局内点,甚至全部为局外点
        maybemodel = n个maybeinliers 拟合出来的可能符合要求的模型
        alsoinliers = emptyset #满足误差要求的样本点,开始置空
        for (每一个不是maybeinliers的样本点)
        {
            if 满足maybemodel即error < t
                将点加入alsoinliers 
        }
        if (alsoinliers样本点数目 > d) 
        {
            %有了较好的模型,测试模型符合度
            bettermodel = 利用所有的maybeinliers 和 alsoinliers 重新生成更好的模型
            thiserr = 所有的maybeinliers 和 alsoinliers 样本点的误差度量
            if thiserr < besterr
            {
                bestfit = bettermodel
                besterr = thiserr
            }
        }
        iterations++
    }
    return bestfit
    """
    iterations = 0
    bestfit = None
    besterr = np.inf #设置默认值
    best_inlier_idxs = None
    while iterations < k:
        maybe_idxs, test_idxs = random_partition(n, data.shape[0])
        print ('test_idxs = ', test_idxs)
        maybe_inliers = data[maybe_idxs, :] #获取size(maybe_idxs)行数据(Xi,Yi)
        test_points = data[test_idxs] #若干行(Xi,Yi)数据点
        maybemodel = model.fit(maybe_inliers) #拟合模型
        test_err = model.get_error(test_points, maybemodel) #计算误差:平方和最小
        print('test_err = ', test_err <t)
        also_idxs = test_idxs[test_err < t]
        print ('also_idxs = ', also_idxs)
        also_inliers = data[also_idxs,:]
        if debug:
            print ('test_err.min()',test_err.min())
            print ('test_err.max()',test_err.max())
            print ('numpy.mean(test_err)',numpy.mean(test_err))
            print ('iteration %d:len(alsoinliers) = %d' %(iterations, len(also_inliers)) )
        # if len(also_inliers > d):
        print('d = ', d)
        if (len(also_inliers) > d):
            betterdata = np.concatenate( (maybe_inliers, also_inliers) ) #样本连接
            bettermodel = model.fit(betterdata)
            better_errs = model.get_error(betterdata, bettermodel)
            thiserr = np.mean(better_errs) #平均误差作为新的误差
            if thiserr < besterr:
                bestfit = bettermodel
                besterr = thiserr
                best_inlier_idxs = np.concatenate( (maybe_idxs, also_idxs) ) #更新局内点,将新点加入
        iterations += 1
    if bestfit is None:
        raise ValueError("did't meet fit acceptance criteria")
    if return_all:
        return bestfit,{'inliers':best_inlier_idxs}
    else:
        return bestfit
 
 
def random_partition(n, n_data):
    """return n random rows of data and the other len(data) - n rows"""
    all_idxs = np.arange(n_data) #获取n_data下标索引
    np.random.shuffle(all_idxs) #打乱下标索引
    idxs1 = all_idxs[:n]
    idxs2 = all_idxs[n:]
    return idxs1, idxs2
 
class LinearLeastSquareModel:
    #最小二乘求线性解,用于RANSAC的输入模型    
    def __init__(self, input_columns, output_columns, debug = False):
        self.input_columns = input_columns
        self.output_columns = output_columns
        self.debug = debug
    
    def fit(self, data):
		#np.vstack按垂直方向（行顺序）堆叠数组构成一个新的数组
        A = np.vstack( [data[:,i] for i in self.input_columns] ).T #第一列Xi-->行Xi
        B = np.vstack( [data[:,i] for i in self.output_columns] ).T #第二列Yi-->行Yi
        x, resids, rank, s = sl.lstsq(A, B) #residues:残差和
        return x #返回最小平方和向量   
 
    def get_error(self, data, model):
        A = np.vstack( [data[:,i] for i in self.input_columns] ).T #第一列Xi-->行Xi
        B = np.vstack( [data[:,i] for i in self.output_columns] ).T #第二列Yi-->行Yi
        B_fit = sp.dot(A, model) #计算的y值,B_fit = model.k*A + model.b
        err_per_point = np.sum( (B - B_fit) ** 2, axis = 1 ) #sum squared error per row
        return err_per_point
 
def test():
    #生成理想数据
    n_samples = 500 #样本个数
    n_inputs = 1 #输入变量个数
    n_outputs = 1 #输出变量个数
    A_exact = 20 * np.random.random((n_samples, n_inputs))#随机生成0-20之间的500个数据:行向量
    perfect_fit = 60 * np.random.normal( size = (n_inputs, n_outputs) ) #随机线性度，即随机生成一个斜率
    B_exact = sp.dot(A_exact, perfect_fit) # y = x * k
 
    #加入高斯噪声,最小二乘能很好的处理
    A_noisy = A_exact + np.random.normal( size = A_exact.shape ) #500 * 1行向量,代表Xi
    B_noisy = B_exact + np.random.normal( size = B_exact.shape ) #500 * 1行向量,代表Yi
 
    if 1:
        #添加"局外点"
        n_outliers = 100
        all_idxs = np.arange( A_noisy.shape[0] ) #获取索引0-499
        np.random.shuffle(all_idxs) #将all_idxs打乱
        outlier_idxs = all_idxs[:n_outliers] #100个0-500的随机局外点
        A_noisy[outlier_idxs] = 20 * np.random.random( (n_outliers, n_inputs) ) #加入噪声和局外点的Xi
        B_noisy[outlier_idxs] = 50 * np.random.normal( size = (n_outliers, n_outputs)) #加入噪声和局外点的Yi
    #setup model 
    all_data = np.hstack( (A_noisy, B_noisy) ) #形式([Xi,Yi]....) shape:(500,2)500行2列
    input_columns = range(n_inputs)  #数组的第一列x:0
    output_columns = [n_inputs + i for i in range(n_outputs)] #数组最后一列y:1
    debug = False
    model = LinearLeastSquareModel(input_columns, output_columns, debug = debug) #类的实例化:用最小二乘生成已知模型
 
    linear_fit,resids,rank,s = sp.linalg.lstsq(all_data[:,input_columns], all_data[:,output_columns])
    
    #run RANSAC 算法
    ransac_fit, ransac_data = ransac(all_data, model, 50, 1000, 7e3, 300, debug = debug, return_all = True)
 
    if 1:
        import pylab
 
        sort_idxs = np.argsort(A_exact[:,0])
        A_col0_sorted = A_exact[sort_idxs] #秩为2的数组
 
        if 1:
            pylab.plot( A_noisy[:,0], B_noisy[:,0], 'k.', label = 'data' ) #散点图
            pylab.plot( A_noisy[ransac_data['inliers'], 0], B_noisy[ransac_data['inliers'], 0], 'bx', label = "RANSAC data" )
        else:
            pylab.plot( A_noisy[non_outlier_idxs,0], B_noisy[non_outlier_idxs,0], 'k.', label='noisy data' )
            pylab.plot( A_noisy[outlier_idxs,0], B_noisy[outlier_idxs,0], 'r.', label='outlier data' )
 
        pylab.plot( A_col0_sorted[:,0],
                    np.dot(A_col0_sorted,ransac_fit)[:,0],
                    label='RANSAC fit' )
        pylab.plot( A_col0_sorted[:,0],
                    np.dot(A_col0_sorted,perfect_fit)[:,0],
                    label='exact system' )
        pylab.plot( A_col0_sorted[:,0],
                    np.dot(A_col0_sorted,linear_fit)[:,0],
                    label='linear fit' )
        pylab.legend()
        pylab.show()
 
if __name__ == "__main__":
    test()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:121: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:110: DeprecationWarning: scipy.dot is deprecated and will be removed in SciPy 2.0.0, use numpy.dot instead


test_idxs =  [433 273 290  99 112 374 251 362  52 383  26 163 479 360 268 443  97 284
 240 298 315 446 340 288 461 295 115  33 142  36 154 304 274 463 363 260
 376 483 453 150 416 246 473 499 209 465 153 344 107 470  64 138 444  19
 200  71 111 377 373 156 186 102 127 216  55 278 405 452 146 355 257 423
 348 481 346  95 271  32 359  10 166 117 419 250  35   4  79 441  77  12
 368  40 236 455 184 404 353 226 429 106 183 233  73 322 103 208  30 239
 193 320  85 254 140 467  45 220  20 180 417 176  58  91  69 341 172 187
 158 130 356 302 170 495 321  25 412 457  27 215  70 287 143 478 406 204
 100 409 464 352  65 266 456 263 491  83 192 203 386 438 128 397 414  66
  47  84 267 484  34 122  57 303 294 436 325 231 155 189 136  29 393  98
 451  92 175 449 135 330 157 366  78   6 476  62 211 174 223 258 324 475
  80 293 460 243 399 205  13 197 425 389 349 145 238 177 151  46 118 498
 311 445  90 190 259 431  17 428 398 426 217  44 104   5 450 133   1 435
 422 361 141 275 317  81 120  68 167 3

d =  300
test_idxs =  [ 34 482  30  31 165 312 451 457  81  75  70 267 392 126 403 394 212  91
 107 458 346 321 266 249 399 440 201 433  16 491 381 188 338 111 233 395
 292 360  27 447 420 187 326 353 278 235 272 379  65   4 151 490 189 268
  45  68 362  25  67 470 369 314 431 133  60 446 391 347 412 478 219 226
 404 351 450 432 220 350  54  93 251 181 434  28 129 214 148  96 335 427
  52  41 421 425 153 205 288 295 279  66 383 453  44 115 225 418   8 466
 108 154 437 340 485 376 155 498 323 442 258  71  87 337 430 463 339  58
 112 417 285 255 158 264  20 330 443 178 265 163  40 436 297 456  14 373
 396 479 159 243   2 454  77  79 146  24 203 101  33 209 428 386 423 462
  63 208 316 113 358  56  92 114 131 476 199  61 132 223 100 136  82 341
 180 120 487 332 402 493  99 448  47 419 193 398 363 127 455  26 275 481
  15 296  38 273 166 492 191 161 168 253 307 182 144 464 310 488 283 494
 465 271 336 231 343 134 306   3 256 170 284 384 320  32 409 238 263 227
 157 484 239  51 177 349 401 

d =  300
test_idxs =  [437 393 181 387 225 310  66 413  97 293 239 251  77 481   9 135 421 356
 428 275 210 414  93 278 445 336 347 149  60 221 281 385 362 237 282 258
  70 486  78 271 152  26 483  80 179  75 497 136 467 106 446 484 351 441
 366 308 371 102  19 305 334 469 463 443 192  73 262 456 329  63  81 323
 376 300  39  41 415  58 301  14 464 327  90  20 482 188 447 286 426 195
 454 338 405 185 394 146 254 208 302   7 341 153 246 296 164 189 150 283
  83 253 379 318 166 333 272 201 403   1 422 139 386 412 306 355 184 399
   4 205 191  45 142 438 214 440 268  46  57 488 274  10 194 487 273 269
 400 206  76  88 382 103 381 460 396 468 471 472  13 197 285 259 470 495
 256  52  96 473  48 219  40  27 111  86 122 158 252 461 368 137 263 427
  55  84 109 326 369 248 316 352  36 303 474 250 174 113  33  30 395 134
 377 325 200  12  67 375 344 264 128  98 120 297 218  32 245  35 157 455
  23 126 182 409  49 304 115 312 291 114 169 222 496 270 307 374  72 121
 317 332 401 345 453 110 290 

d =  300
test_idxs =  [499 297 398  64 453  84 458 144  98 461 102  48 215  10 360 424 296 406
 148 311 489 118 361 387 229 312 350 335 232  54 353 478 492 494 135 252
 421 101 338 432  25 256 333 320 347 346  85 288 285 309  95   8 337 385
 490  81 305 220 434   1 483 497 113 449 339 145 238 153 143 174 493  22
 351  72 223 189 375 149 206  33 476 313 356 306 287 321 329 157 465 161
 181  11  17 115 303 310  82 344 221 261 455 460  70 370 314  55 328  27
 433 164 386  56 437  13 124 116 262 230 443 131 132 169 123 484 431 322
  28 376 279 452 188 412  83 119 423 217   6 388 378 395  60 260 203 399
 126 445 171 440 120 258 183  62 332 242 129 444 400  99 470 274 117 323
 298 377 165  74 435  88 404 212 108 317 246 127 255 151 270 426 196 380
 414  79 359 191 481 103 290 330 411 125  45 121 417 462 233 403 278  68
 130 448  78 134  63  29 112 147 422  71 286 397  51 307 280 142 198 175
 457  97  53 410 154 358 486 364  36  16 463 402 227  26 413 263 477 408
 373 369 250 128 363 194 208 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# 3.实现均值哈希和插值哈希

In [4]:
import cv2
import numpy as np
 
#均值哈希算法
def aHash(img):
    #缩放为8*8
    img=cv2.resize(img,(8,8),interpolation=cv2.INTER_CUBIC)
    #转换为灰度图
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #s为像素和初值为0，hash_str为hash值初值为''
    s=0
    hash_str=''
    #遍历累加求像素和
    for i in range(8):
        for j in range(8):
            s=s+gray[i,j]
    #求平均灰度
    avg=s/64
    #灰度大于平均值为1相反为0生成图片的hash值
    for i in range(8):
        for j in range(8):
            if  gray[i,j]>avg:
                hash_str=hash_str+'1'
            else:
                hash_str=hash_str+'0'            
    return hash_str
 
#差值算法
def dHash(img):
    #缩放8*9
    img=cv2.resize(img,(9,8),interpolation=cv2.INTER_CUBIC)
    #转换灰度图
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    hash_str=''
    #每行前一个像素大于后一个像素为1，相反为0，生成哈希
    for i in range(8):
        for j in range(8):
            if   gray[i,j]>gray[i,j+1]:
                hash_str=hash_str+'1'
            else:
                hash_str=hash_str+'0'
    return hash_str
 
#Hash值对比
def cmpHash(hash1,hash2):
    n=0
    #hash长度不同则返回-1代表传参出错
    if len(hash1)!=len(hash2):
        return -1
    #遍历判断
    for i in range(len(hash1)):
        #不相等则n计数+1，n最终为相似度
        if hash1[i]!=hash2[i]:
            n=n+1
    return n
 
img1=cv2.imread('rabbit01.png')
img2=cv2.imread('rabbit02.png')
hash1= aHash(img1)
hash2= aHash(img2)
print(hash1)
print(hash2)
n=cmpHash(hash1,hash2)
print('均值哈希算法相似度：',n)
 
hash1= dHash(img1)
hash2= dHash(img2)
print(hash1)
print(hash2)
n=cmpHash(hash1,hash2)
print('差值哈希算法相似度：',n)

1111111111101011111000111100001111110011101100111110001111111111
1111111111110111110001111100001111000001111000011111001111111111
均值哈希算法相似度： 12
0010011100100100101100000101100011010000010110001111000000110101
0000000000010000010100000111100001111100011101000010100000000000
差值哈希算法相似度： 26
